In [ ]:
from bs4 import BeautifulSoup
from pprint import pprint
import json
import requests

In [ ]:
def getMovieName(movieName):
    #
    # 영화 검색화면에서 첫번째 영화의 코드를 가져온다.
    # 
    
    # 영화 이름이 검색되지않으면 None return
    try:
        SEARCH_URL = "https://watcha.com/ko-KR/search?"
        params = {
                "query" : movieName,
            }
        resp = requests.get(SEARCH_URL, params = params)
        soup = BeautifulSoup(resp.content, "html.parser")
        
        return soup.find("li", class_ = "css-106b4k6-Self").a['href'].split("/")[-1]
    except:
        return None

In [ ]:
def setURL(movieName, category):
    #
    # getMovieName()에서 받아온 코드를 이용하여 URL을 만들어준다.
    # WATCHA_URL: overview를 위한 URL
    # WATCHA_API_URL: api를 사용하여 댓글을 가져오기 위한 URL
    #
    WATCHA_URL = "https://watcha.com/ko-KR/contents"
    WATCHA_API_URL = "https://api.watcha.com/api/contents"
    
    # 영화 이름이 검색되지않아서 영화 코드를 받지못했다면 return None
    try:
        MOVIE_CODE = getMovieName(movieName)
    except:
        return None
    
    # 카테고리 분류
    categoryDic = {
        'overview' : 'overview',
        'comments' : 'comments'    
    }
    
    # 헤더
    headers = {
        'Referer' : f"https://watcha.com/ko-KR/contents/{MOVIE_CODE}/{categoryDic[category]}",
        'x-watcha-client' : 'watcha-WebApp',
        'x-watcha-client-language' : 'ko',
        'x-watcha-client-region' : 'KR',
        'x-watcha-client-version' : "1.0.0"
    }

    requestList =[ WATCHA_URL, MOVIE_CODE, categoryDic[category] ]
    apiList = [ WATCHA_API_URL, MOVIE_CODE, categoryDic[category] ]

    REQUEST_URL = "/".join(requestList)
    API_URL = "/".join(apiList)
    
    print("movie code 변환 완료")
    
    # 입력받은 category가 'comments'인지 'overview'인지 확인 후 return
    return (API_URL, headers) if(category =='comments') else (REQUEST_URL, headers)

In [ ]:
def getReviewList(movieName):
    #
    # 리뷰 리스트를 가져오는 함수
    # 영화가 존재하지않을때 return None
    #
    
    try:
        API_URL, headers = setURL(movieName, 'comments')
    except:
        return None
    print("API_URL 세팅 완료")
    
    # 파라미터, api를 통해 리뷰를 가져오기위해 요청할때 같이 보내는 파라미터
    # 기존에는 스크롤을 할때 리뷰가 3개씩 추가
    params = {
        'filter' : 'all',
        'order' : 'popular',
        'page' : 1,
        'size' : 20
    }

    # 페이로드 
    # 로그인 정보, 바꿔야함
    datas = {
        "email" : "email",
        "password" : "password"
    }
    
    # return할 결과 list 
    reviewList = []
    
    # review 갯수 
    cntReview = 0
    
    # 로그인 정보
    session = requests.session()
    session.post("https://api.watcha.com/api/sessions", headers = headers, data = datas)

    # api로 받은 값을 list에 저장
    print("review crawling 시작")
    while(True):
        resp = session.get(API_URL, headers = headers, params = params)
        reviewJson = resp.json()
        if(len(reviewJson['result']['result']) == 0):
            break
        for i in reviewJson['result']['result']:
            reviewList.append(i)
            cntReview += 1
        params['page'] += 1
        
        # 100개씩 cnt
        if(cntReview%100) == 0:
            print(f"{cntReview}개")
        
    print(f"review {len(reviewList)}개 crawling 완료")
    return reviewList

In [ ]:
def getOverview(movieName):
    #
    # 영화 정보를 가져오는 함수
    # 영화가 존재하지않을때 return None
    #
    
    try:
        REQUEST_URL, headers = setURL(movieName, 'overview')
    except:
        return None
    print("REQUEST_URL 세팅 완료")
    
    # return할 결과 dict
    overviewDict = {}
    
    resp = requests.get(REQUEST_URL, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")
    ulContents = soup.find("ul", class_ = "css-1w7dpv-VisualUl-DescriptionUl")
#     ulContents.select("dl dt")
#     ulContents.select("dl dd")


    # 원제, 제작 연도, 국가, 장르, 상영시간, 내용
    print("overview crawling 시작")
    for i in range(len(ulContents.select("dl dt"))):
        key = ulContents.select("dl dt")[i].text
        val = ulContents.select("dl dd")[i].text
        overviewDict[key] = val if(key != '제작 연도') else int(val)
    print("overview crawling 종료")
    
    return overviewDict

In [ ]:
movieName = input("영화 제목을 입력하세요.\n")
output = getReviewList(movieName)
try:
    output.insert(0, getOverview(movieName))
except:
    print("영화를 찾지 못했습니다.\n올바른 영화이름을 입력해주세요.")

In [ ]:
pprint(output)


In [ ]:
with open(f"{output}.json", 'w', encoding='utf-8') as file :
        json.dump(review, file, ensure_ascii=False, indent='\t')